In [22]:
import sys
import os
import warnings

sys.path.append(os.path.abspath("modules/"))
sys.path.append(os.path.abspath("fe_modules/"))
sys.path.append(os.path.abspath("seq2seq_modules/"))

os.environ['OPENBLAS_NUM_THREADS'] = '1'
warnings.filterwarnings('ignore')

In [23]:
import pandas as pd
import numpy as np
import time
import pyarrow.parquet as pq
import polars as pl
import scipy
import implicit
import bisect
import sklearn.metrics as m
from catboost import CatBoostClassifier, CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.calibration import calibration_curve, CalibratedClassifierCV

In [25]:
import importlib

import modules
import fe_modules

importlib.reload(modules)
importlib.reload(fe_modules)
importlib.reload(fe_modules.aggregates)

from modules.memory_utils import polars_reduce_mem_usage

In [11]:
LOCAL_DATA_PATH = './context_data/'
SPLIT_SEED = 42
DATA_FILE = 'competition_data_final_pqt/'
TARGET_FILE = 'competition_target_pqt/'

In [12]:
df = polars_reduce_mem_usage(
    pl.read_parquet(
        f'context_data/competition_data_final_pqt/part-00000-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet'
    ))
df

Memory usage of dataframe is 6059.32 MB
Memory usage of dataframe is 5.92 GB
utf
utf
utf
utf
utf
utf
utf
flo
dat
utf
int
int
Memory usage after optimization is: 5592.42 MB
Memory usage  after optimization is 5.46 GB
Decreased by 7.7%


region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,date,part_of_day,request_cnt,user_id
str,str,str,str,str,str,str,f32,date,str,i8,i32
"""Краснодарский ...","""Краснодар""","""Apple""","""iPhone 7""","""ad.adriver.ru""","""smartphone""","""iOS""",20368.0,2022-06-15,"""morning""",1,45098
"""Краснодарский ...","""Краснодар""","""Apple""","""iPhone 7""","""apple.com""","""smartphone""","""iOS""",20368.0,2022-06-19,"""morning""",1,45098
"""Краснодарский ...","""Краснодар""","""Apple""","""iPhone 7""","""avatars.mds.ya...","""smartphone""","""iOS""",20368.0,2022-06-12,"""day""",1,45098
"""Краснодарский ...","""Краснодар""","""Apple""","""iPhone 7""","""googleads.g.do...","""smartphone""","""iOS""",20368.0,2022-05-16,"""day""",1,45098
"""Краснодарский ...","""Краснодар""","""Apple""","""iPhone 7""","""googleads.g.do...","""smartphone""","""iOS""",20368.0,2022-05-30,"""day""",1,45098
"""Краснодарский ...","""Краснодар""","""Apple""","""iPhone 7""","""i.ytimg.com""","""smartphone""","""iOS""",20368.0,2022-03-29,"""evening""",2,45098
"""Краснодарский ...","""Краснодар""","""Apple""","""iPhone 7""","""icloud.com""","""smartphone""","""iOS""",20368.0,2022-03-17,"""morning""",1,45098
"""Краснодарский ...","""Краснодар""","""Apple""","""iPhone 7""","""m.avito.ru""","""smartphone""","""iOS""",20368.0,2022-05-19,"""morning""",1,45098
"""Краснодарский ...","""Краснодар""","""Apple""","""iPhone 7""","""relap.io""","""smartphone""","""iOS""",20368.0,2022-03-29,"""night""",1,45098


# Feature engeneering

In [19]:
from fe_modules.aggregates import get_agg_count, \
                                  get_agg_sum, \
                                  get_agg_mode, \
                                  get_price_of_all_cpes, \
                                  get_agg_n_unique

In [20]:
counts = get_agg_count(df)
counts.head(5)

user_id,user_id_group_count
i32,u32
4,591
16,1231
18,1125
26,1018
27,2670


In [27]:
total_price = get_price_of_all_cpes(df)
total_price.head(5)

user_id,total_price
i32,f32
132432,7990.0
60408,35931.0
1968,18378.0
304920,49990.0
62200,15845.0
